# Credit default
__Note on how to use this notebook:__ <br>
1) Save the notebook to disk. <br>
2) Save the [data set](https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls) to the same folder that this notebook was saved in. 

[Data description](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#) <br>




# Ideas:
Model-function Keras, layers in for loop, layer number as  input.

In [1]:
import pandas as pd
import os
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

The following runs the data preperation that is used for all models.

We scale all features by Sci-Kit learn's standard scaler. The standard scalars subtracts the mean, so that the means of the standardized variables equal zero. Furthermore the standard scaler divides the feautres by their respective variances, so that the variances of the standardized features equals one.

In [2]:
# Reading file into data frame
cwd = os.getcwd()
filename = cwd + '/default of credit card clients.xls'
nanDict = {}
df = pd.read_excel(filename, header=1, skiprows=0, index_col=0, na_values=nanDict)

df.rename(index=str, columns={"default payment next month": "defaultPaymentNextMonth"}, inplace=True)

# Features and targets 
X = df.loc[:, df.columns != 'defaultPaymentNextMonth'].values
y = df.loc[:, df.columns == 'defaultPaymentNextMonth'].values

# Categorical variables to one-hot's
onehotencoder = OneHotEncoder(categorical_features = [3])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:] 

# Train-test split
trainingShare = 0.5 
XTrain, XTest, yTrain, yTest=train_test_split(X, y, train_size=trainingShare, \
                                              test_size = 1-trainingShare)

# Input Scaling
sc = StandardScaler()
XTrain = sc.fit_transform(XTrain)
XTest = sc.transform(XTest)

# One-hot's of the target vector
Y_train_onehot, Y_test_onehot = to_categorical(yTrain), to_categorical(yTest)

# Descriptive information
print('Number of empty elements in data: ', df.isnull().values.any())
print('Observations: ', df.shape[0])
print('Percentage defaults: ', df['defaultPaymentNextMonth'].astype(bool).sum(axis=0)/df.shape[0]*100)

Number of empty elements in data:  False
Observations:  30000
Percentage defaults:  22.12


This is not the same number of observations as in Yeh and Lien (2009). Yeh and Lien (2009) have 25 000 observations. However, we have the same number of observations as in Pyzhov and Pyzhov (2017), which is said to use the same dataset as Yeh and Lien (2009). 

The percentage of individuals with default is the same as in the Yeh and Lien (2009).


# Logistic regression
We apply Sci-Kit learn's logistic regression method for performing classification of default and non-defaulting customers. It is possible to use regularization for the logistic regression. Regularization has the potential to reduce overfitting. We will apply Sci-Kit learn's Grid search function for identifying the optimal regularization value. The optimal regularization parameter is determined by the accuracy score on test sets applying K-fold cross validation.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

lmbdas=np.logspace(-5,7,13)
parameters = [{'C': 1./lmbdas}]
scoring = ['accuracy', 'roc_auc']
logReg = LogisticRegression()
gridSearch = GridSearchCV(logReg, parameters, cv=5, scoring=scoring, refit='accuracy') 
# "refit" gives the metric used deciding best model. 
# See more http://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html
gridSearch.fit(XTrain, yTrain)

/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': array([1.e+05, 1.e+04, 1.e+03, 1.e+02, 1.e+01, 1.e+00, 1.e-01, 1.e-02,
       1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07])}],
       pre_dispatch='2*n_jobs', refit='accuracy',
       return_train_score='warn', scoring=['accuracy', 'roc_auc'],
       verbose=0)

In [8]:
print('Best parameter: ',gridSearch.best_params_, '\n')
results = gridSearch.cv_results_
for trainAccuracy, testAccuracy, trainAUC, testAUC, parameter in zip(results['mean_train_accuracy'], 
                                                                     results['mean_test_accuracy'],
                                                                     results['mean_train_roc_auc'],
                                                                     results['mean_test_roc_auc'],
                                                                     results['params']):
    print('Parameter ', parameter, 'Train accuracy %.3f Test accuracy %.3f Train AUC %.3f Test AUC %.3f '
          %(trainAccuracy, testAccuracy, trainAUC, testAUC), parameter)

Best parameter:  {'C': 0.1} 

Parameter  {'C': 99999.99999999999} Train accuracy 0.811 Test accuracy 0.810 Train AUC 0.724 Test AUC 0.722  {'C': 99999.99999999999}
Parameter  {'C': 10000.0} Train accuracy 0.811 Test accuracy 0.810 Train AUC 0.724 Test AUC 0.722  {'C': 10000.0}
Parameter  {'C': 1000.0} Train accuracy 0.811 Test accuracy 0.810 Train AUC 0.724 Test AUC 0.722  {'C': 1000.0}
Parameter  {'C': 100.0} Train accuracy 0.811 Test accuracy 0.811 Train AUC 0.724 Test AUC 0.722  {'C': 100.0}
Parameter  {'C': 10.0} Train accuracy 0.811 Test accuracy 0.810 Train AUC 0.724 Test AUC 0.722  {'C': 10.0}
Parameter  {'C': 1.0} Train accuracy 0.811 Test accuracy 0.810 Train AUC 0.724 Test AUC 0.722  {'C': 1.0}
Parameter  {'C': 0.1} Train accuracy 0.811 Test accuracy 0.811 Train AUC 0.724 Test AUC 0.721  {'C': 0.1}
Parameter  {'C': 0.01} Train accuracy 0.810 Test accuracy 0.809 Train AUC 0.723 Test AUC 0.721  {'C': 0.01}
Parameter  {'C': 0.001} Train accuracy 0.808 Test accuracy 0.807 Train A

/home/k/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/k/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


We see that in terms of accuracy it does not matter much what the regularization parameter value is. The optimal parameter, among the chosen parameter values, is one, but the difference in test score between the eight first regularization parameter values is practically non-existent.

### Logistic regression: Fitting and testing the model with the best hyperparameter

In [15]:
C = gridSearch.best_params_['C']
logRegFinal = LogisticRegression(C=C, random_state=1)
logRegFinal.fit(XTrain, yTrain)

/home/k/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
print('\n###################  Training  ###############')
yPredTrain = logRegFinal.predict(XTrain)#logReg.predict(XTrain)
yPredTrain = (yPredTrain > 0.5)
cm = confusion_matrix(
    yTrain, yPredTrain) #.argmax(axis=1) Y_train_onehot.argmax(axis=1)
print('\nTraining Confusion matrix: \n', cm)
accScore = accuracy_score(yTrain, yPredTrain)
print('\nTraining Accuracy score: \n', accScore)
print('\nTrain AUC: \n', roc_auc_score(yTrain, yPredTrain))


print('\n###################  Testing  ###############')
yPred = logRegFinal.predict(XTest)#logReg.predict(XTest)
yPred = (yPred > 0.5)
cm = confusion_matrix(
    yTest, yPred) 
print('\nTest Confusion matrix: \n', cm)
accScore = accuracy_score(yTest, yPred)
print('\nTest Accuracy score: \n', accScore)
print('\nTest AUC: \n', roc_auc_score(yTest, yPred))


###################  Training  ###############

Training Confusion matrix: 
 [[11401   296]
 [ 2542   761]]

Training Accuracy score: 
 0.8108

Train AUC: 
 0.6025454876099876

###################  Testing  ###############

Test Confusion matrix: 
 [[11375   292]
 [ 2561   772]]

Test Accuracy score: 
 0.8098

Test AUC: 
 0.6032976529846351


The accurcies and AUC's on the testing set is higher the cooresponding best mean numbers on the validation sets from the K-fold cross validation. We see that only about $1/3$ of the defaults get predicted.

## Keras NN
We will now perform classification by deep neural networks. Keras is used. 


### Grid search
We will apply Sci-Kit learn's grid search function in order to determine the optimal combination of hyperparameters.

In [3]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV



def createModel(neurons =50, hiddenLayers = 2):
    model = tf.keras.Sequential()
    neuronsPerLayer = neurons // (hiddenLayers + 1)
    model.add(tf.keras.layers.Dense(neuronsPerLayer, activation='relu', input_dim=XTrain.shape[1]))
    for i in range(hiddenLayers):
        model.add(tf.keras.layers.Dense(neuronsPerLayer, activation='relu'))
    model.add(tf.keras.layers.Dense(Y_train_onehot.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=createModel, verbose=0)

neurons = [20, 100, 200, 500]#, 50, 75]
batch_size = [5, 10, 100]##, 40, 60, 80, 100]
hiddenLayers = [1,3, 5]
parameterGrid = [{'neurons': neurons, 'hiddenLayers': hiddenLayers},
                 {'batch_size': batch_size}]
folds = 3

grid = GridSearchCV(estimator=model, cv=folds, param_grid=parameterGrid, n_jobs=-1)

epochs = 10
grid_result = grid.fit(XTrain, Y_train_onehot, epochs=epochs)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.815400 using {'hiddenLayers': 1, 'neurons': 200}
0.813133 (0.002451) with: {'hiddenLayers': 1, 'neurons': 20}
0.812067 (0.003594) with: {'hiddenLayers': 1, 'neurons': 100}
0.815400 (0.002993) with: {'hiddenLayers': 1, 'neurons': 200}
0.814533 (0.003021) with: {'hiddenLayers': 1, 'neurons': 500}
0.801067 (0.006543) with: {'hiddenLayers': 3, 'neurons': 20}
0.810067 (0.006614) with: {'hiddenLayers': 3, 'neurons': 100}
0.810533 (0.004620) with: {'hiddenLayers': 3, 'neurons': 200}
0.813733 (0.004878) with: {'hiddenLayers': 3, 'neurons': 500}
0.784667 (0.004823) with: {'hiddenLayers': 5, 'neurons': 20}
0.802067 (0.004521) with: {'hiddenLayers': 5, 'neurons': 100}
0.812267 (0.004485) with: {'hiddenLayers': 5, 'neurons': 200}
0.811333 (0.003920) with: {'hiddenLayers': 5, 'neurons': 500}
0.813067 (0.003348) with: {'batch_size': 5}
0.814400 (0.003374) with: {'batch_size': 10}
0.797800 (0.010999) with: {'batch_size': 100}


We see that the best combination of the chosen number of hidden layers and neuron numbers is one hidden layer and two hundred neurons. 

A batch size of 10 is the best among the chosen batch sizes.

Next we apply the optimal combination of batch size and neuron number from the crossvalidation train a model on the full training set. The model based on the full training set will then be applied to measure the accuracy on predictions on the test set.  

### FItting the best model: early stopping
We see from the above that the validation accuracy declines for the highest number of neurons. The decline in validation accuracy is a sign of overfitting. In order to avoid overfitting we use a methods for "early stopping". Early stopping stops the simulations when the validation set performance has dropped a user given number of times in a row. In order for the model to be able to escape local minima, we allo the validation accuracy to drop a few times before breaking. 

In [9]:
hiddenLayers, neurons =  grid_result.best_params_['hiddenLayers'], grid_result.best_params_['neurons']
batch_size = 10

model = KerasClassifier(build_fn=createModel, verbose=0, neurons =neurons, hiddenLayers = hiddenLayers)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_acc',
                                             min_delta=0,
                                             patience=2, # argument represents the number of epochs before stopping once your loss starts to increase (stops improving)
                                             verbose=0, 
                                             mode='auto')]#,
                                             #restore_best_weights=True)] # Use best model
history = model.fit(XTrain,
                        Y_train_onehot,
                        epochs=15, 
                        batch_size=batch_size,
                        validation_data=[XTest, Y_test_onehot],
                        callbacks = callbacks)

print('Number of epochs before early stopping: ', len(history.history['loss']))

Number of epochs before early stopping:  4


Confusion matrices, accuracy scores and AUC-numbers:

In [10]:
print('\n###################  Training  ###############')
yPredTrain = model.predict(XTrain)
yPredTrain = (yPredTrain > 0.5)
cm = confusion_matrix(
    yTrain, yPredTrain) 
print('\nTraining Confusion matrix: \n', cm)
accScore = accuracy_score(yTrain, yPredTrain)
print('\nTraining Accuracy score: \n', accScore)
print('\nTrain AUC: \n', roc_auc_score(yTrain, yPredTrain))


print('\n###################  Testing  ###############')
yPred = model.predict(XTest)
yPred = (yPred > 0.5)
cm = confusion_matrix(
    yTest, yPred) 
print('\nTest Confusion matrix: \n', cm)
accScore = accuracy_score(yTest, yPred)
print('\nTest Accuracy score: \n', accScore)
print('\nTest AUC: \n', roc_auc_score(yTest, yPred))


###################  Training  ###############

Training Confusion matrix: 
 [[11172   525]
 [ 2190  1113]]

Training Accuracy score: 
 0.819

Train AUC: 
 0.6460415453879857

###################  Testing  ###############

Test Confusion matrix: 
 [[11129   538]
 [ 2206  1127]]

Test Accuracy score: 
 0.8170666666666667

Test AUC: 
 0.6460104225902148


Only about a third of the defaulting customers are correctly predicted.

Ok accuracy. Not impressing performance when it compes to predicting the defaults. Only a 3rd of the defaults in the test set is correctly predicted. <br>

Yeh and Lien (2009) get: <br>
Training accuracy: 0.81<br>
Training AUC: 0.55 <br>
Testing accuracy: 0.83<br>
Testing AUC: 0.54 <br>

We see that we get the accuracy the the same level as Yeh and Lien (2009), but the AUC is better. However, it is unclear wheteher it acutally is AUC Yeh and Lien (2009) applies, as they call it area ratio.<br>

Pyzhov and Pyzhov (2017) get about the same acuracy as we do, but higher AUCs.

# Principal component analyses
We will explore the effects of training the network with principcal components. For many of the networks we do not expect the introduction of principcal components to improve the performance considerably. The difference between training and testing accuracy is small for many of the networks, indicating that there is little overfitting. Maybe for the network setuos where there are larger deviations between training and testing accuracy, the networks with mutiple hidden layers, there can be gains from introducing principcal components. 

We use the principal components that explain 95 per cent of total variance.

In [31]:
from sklearn.decomposition import PCA

'''
pca = PCA(n_components = 4)
pca.fit_transform(X)
#pca.components_.T[:,0] # Displays component
pca.explained_variance_ratio_
'''

pca = PCA(n_components = 0.95)
Xreduced = pca.fit_transform(X)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))

[0.84870179 0.04871288 0.02708606 0.01606241 0.0144179 ]
0.9549810379752075


In [44]:
trainingShare = 0.5 
XTrain, XTest, yTrain, yTest=train_test_split(Xreduced, y, train_size=trainingShare, \
                                              test_size = 1-trainingShare)
Y_train_onehot, Y_test_onehot = to_categorical(yTrain), to_categorical(yTest)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu', input_dim=XTrain.shape[1]))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(Y_train_onehot.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
print(model.summary())

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             min_delta = 0,
                                             patience=5,
                                             verbose=0,
                                             mode='auto')]


history = model.fit(XTrain,
                    Y_train_onehot, 
                    epochs=100, 
                    batch_size=30,
                    validation_data=[XTest, Y_test_onehot],
                    callbacks = callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 50)                300       
_________________________________________________________________
dense_50 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_51 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_52 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_53 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_54 (Dense)             (None, 2)                 102       
Total params: 10,602
Trainable params: 10,602
Non-trainable params: 0
_________________________________________________________________
None
T

We see that both training and testing accuracy is reduced when using Principal components as predictors instead of all of the original features. However, we also observe that the difference between training and validation accuracy is smaller when using principal components as predictors instead of all the features from the original data set.

## Kernel PCA
We will try Sci-Kit learn's kernel PCA method. Kernel PCA, kPca, is a PCA-method that allows for non-linearity. <mark> More about this!
    


In [60]:
from sklearn.decomposition import KernelPCA, TruncatedSVD
kPCA = KernelPCA(n_components=2, kernel='rbf', gamma=10) #0.04
#kPCA = TruncatedSVD(n_components=5, algorithm='arpack')
X_reduced = kPCA.fit_transform(X)
print(kPCA.explained_variance_ratio_)
print(np.sum(kPCA.explained_variance_ratio_))

MemoryError: 

kPCA did not work. It gave memory error.

# Support Vector Machines (SVM)
We will now apply the SVM classifier to make the classification. We start by running the standard SVM estimator, and then we try alternative methods that potentially increase accuracy in the presence of non-linearity in the data. By "non-linearity" we mean that the labels cannot be separated by a linear classification plane (line in 2D, 2D plane in 3D, hyperplane for higher dimensions than 3).

# References
Yeh, I-C. and Lien, C-h. (2009). The comparisons of data mining techniques for the predictive
accuracy of probability of default of credit card clients. <br>
_Expert Systems with Applications_ 36 (2009) 2473–2480. <br>
https://bradzzz.gitbooks.io/ga-seattle-dsi/content/dsi/dsi_05_classification_databases/2.1-lesson/assets/datasets/DefaultCreditCardClients_yeh_2009.pdf

Pyzhov, V. and Pyzhov, S. (2017). Comparison of methods of data mining
techniques for the predictive accuracy. _MPRA Paper_ No. 79326. <br>
https://mpra.ub.uni-muenchen.de/79326/1/MPRA_paper_79326.pdf
